## Step -1 Business Problem Understanding

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Step -2 Data Understanding

#### Load data & Understand every variable

In [2]:
df = pd.read_csv('insurance.csv')
df.head()

,age,sex,bmi,children,smoker,region,expenses
0,19,female,27.9,0,yes,southwest,16884.92
1,18,male,33.8,1,no,southeast,1725.55
2,28,male,33.0,3,no,southeast,4449.46
3,33,male,22.7,0,no,northwest,21984.47
4,32,male,28.9,0,no,northwest,3866.86


#### dataset understanding

In [3]:
df.shape

(1338, 7)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   expenses  1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [5]:
df['sex'].value_counts()

sex
male      676
female    662
Name: count, dtype: int64

In [6]:
df['children'].value_counts()

children
0    574
1    324
2    240
3    157
4     25
5     18
Name: count, dtype: int64

In [7]:
df['smoker'].value_counts()

smoker
no     1064
yes     274
Name: count, dtype: int64

In [8]:
df['region'].value_counts()

region
southeast    364
southwest    325
northwest    325
northeast    324
Name: count, dtype: int64

#### Exploratory Data Analysis

In [9]:
continuous_features = ['age', 'bmi', 'expenses']
discrete_categorical = ['sex', 'smoker', 'region']
discrete_count = ['children']

In [10]:
df[continuous_features].describe()

,age,bmi,expenses
count,1338.000000,1338.000000,1338.000000
mean,39.207025,30.665471,13270.422414
std,14.049960,6.098382,12110.011240
min,18.000000,16.000000,1121.870000
25%,27.000000,26.300000,4740.287500
50%,39.000000,30.400000,9382.030000
75%,51.000000,34.700000,16639.915000
max,64.000000,53.100000,63770.430000


In [11]:
df[discrete_categorical].describe()

,sex,smoker,region
count,1338,1338,1338
unique,2,2,4
top,male,no,southeast
freq,676,1064,364


In [12]:
df[continuous_features].corr()

,age,bmi,expenses
age,1.000000,0.109341,0.299008
bmi,0.109341,1.000000,0.198576
expenses,0.299008,0.198576,1.000000


## Step - 3: Data Preprocessing

#### Data Cleaning

In [13]:
df.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
expenses    0
dtype: int64

In [14]:
df.duplicated().sum()

1

In [15]:
df.drop_duplicates(inplace=True)

In [16]:
df.shape

(1337, 7)

In [17]:
df.drop('region', axis=1, inplace=True)

#### Encoding

In [18]:
# encoding sex column
df['sex'].replace({'female':0, 'male':1}, inplace=True)

# encoding 'smoker' column
df['smoker'].replace({'no':0, 'yes':1}, inplace=True)

#### X&y

In [19]:
X = df.drop('expenses', axis=1)
y = df['expenses']

## Step - 4: Modeling

### findout the best random_state value 

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score

Train = []
Test = []
CV = []

for i in range(0, 100):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)
    model = LinearRegression()
    model.fit(X_train, y_train)

    ypred_train = model.predict(X_train)
    ypred_test = model.predict(X_test)

    Train.append(model.score(X_train, y_train))
    # print('Train R2:', r2_score(y_train, ypred_train))
    Test.append(model.score(X_test, y_test))
    # print('Test R2:', r2_score(y_test, ypred_test))

    CV.append(cross_val_score(model, X_train, y_train, cv=5).mean())


em = pd.DataFrame({'Train':Train, 'Test':Test, 'CV':CV})
gm = em[(abs(em['Train']-em['Test']) <= 0.05) & (abs(em['Test']-em['CV']) <=0.05)]
print('best random state number:', gm[gm['Test']==gm['Test'].max()].index.to_list())

best random state number: [90]


In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=90)

# <font color = aqua> Elasticnet Regresion </font> 

#### ElasticNet Regression with default parameters

In [22]:
from sklearn.linear_model import ElasticNet
enr_base = ElasticNet()
enr_base.fit(X_train, y_train)

print('Intercept:', enr_base.intercept_)
print('coefficient:', enr_base.coef_)

# Predictions
ypred_train = enr_base.predict(X_train)
ypred_test = enr_base.predict(X_test)

# Evaluation
from sklearn.metrics import r2_score
print('Train R2:', r2_score(y_train, ypred_train))
print('Test R2:', r2_score(y_test, ypred_test))
from sklearn.model_selection import cross_val_score
print('CV Score:', cross_val_score(enr_base, X_train, y_train, cv=5).mean())

Intercept: -8323.696325445788
coefficient: [ 260.62233727  332.40307796  310.76451947  355.44135908 5669.32895328]
Train R2: 0.38716955393373964
Test R2: 0.37019647328097105
CV Score: 0.3850791425338946


#### Apply Hyperparameter tuning for ElastiNet Regression

In [23]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet

estimator = ElasticNet()
# Parameters & values
param_grid = {'alpha':[0.1, 0.2, 1, 2, 3, 5, 10], 'l1_ratio':[0.1, 0.5, 0.75, 0.9, 0.95, 1]}
# Identifying the best value of the parameter within given values for the given data
model_hp = GridSearchCV(estimator, param_grid, cv=5, scoring='neg_mean_squared_error')
model_hp.fit(X_train, y_train)
model_hp.best_params_ 

{'alpha': 10, 'l1_ratio': 1}

- for every  α it will check all λ differently (on backend nested for loop will work)
- 'l1_raio:' ==> λ
- when λ = 1 that basically means (100% lasso)
- (so we can say lasso is best out of both lasso and ridge)

#### Rebuilt ElasticNet Model using best hyperparameters

In [24]:
from sklearn.linear_model import ElasticNet
enr_best = ElasticNet(alpha=10, l1_ratio=1)
enr_best.fit(X_train, y_train)

print('Intercept:', enr_best.intercept_)
print('Coefficients:', enr_best.coef_)

# Prediction 
ypred_train = enr_best.predict(X_train)
ypred_test = enr_best.predict(X_test)

# Evaluation
from sklearn.metrics import r2_score
print('Train R2:', r2_score(y_train, ypred_train))
print('Test R2:', r2_score(y_test, ypred_test))
from sklearn.model_selection import cross_val_score
print('CV Score:', cross_val_score(enr_best, X_train, y_train, cv=5).mean())

Intercept: -11955.118584050677
Coefficients: [  263.62506852  -153.60684459   315.44544353   445.81295475
 23746.97713616]
Train R2: 0.7408806662327847
Test R2: 0.7815059773758312
CV Score: 0.7387256662169683


#### Prediction on New Data

In [25]:
input_data = {'age':35,
             'sex':'Male',
             'bmi':31.4,
             'children':5,
             'smoker':'yes',
             'region':'southeast'}
df_test = pd.DataFrame(input_data, index=[0])
df_test

,age,sex,bmi,children,smoker,region
0,35,Male,31.4,5,yes,southeast


In [26]:
df_test.drop('region', axis=1, inplace=True)

In [27]:
# encoding sex column
df_test['sex'].replace({'female':0, 'Male':1}, inplace=True)

# encoding 'smoker' column
df_test['smoker'].replace({'no':0, 'yes':1}, inplace=True)

In [28]:
X5 = df_test

In [29]:
enr_best.predict(X5)

array([32999.18080634])

>>

### Multiple Linear regression
- Test R2: 0.7814
- expenses = 33059.918 (expenses with 78.14% accuarcy)

### Polynomial regression
- Test R2: 0.8112
- expenses = 26684.358 (expenses with 81.12% accuarcy)

### Lasso regression
- Test R2: 0.7812
- expenses = 32630.321 (expenses with 78.12% accuarcy)

### Ridge regression
- Test R2: 0.7813
- expenses = 32840.671 (expenses with 78.13% accuarcy)

### ElasticNet regression
- Test R2: 0.7815
- expenses = 32999.180 (expenses with 78.15% accuarcy)